<a href="https://colab.research.google.com/github/marinavasqr23/Digital-College-Data-Analytics/blob/main/aula03_etl_contratos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de ETL Simplificado com Python

**Contexto de problema**: Imagine que somos consultores de dados e estamos tendo que lidar com vários arquivos de contratos.

**Proposta**: Criar um sistema que ler vários arquivos, faz análise e gera o arquivo consolidado.

Temos:

1. Uma tabela de contrato.
2. Uma tabela de dados de empresas.
3. Uma tabela com dados de datas.

#Setup - Configurando o Ambiente



In [1]:
import google.colab.drive
google.colab.drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore') # Ignorar os avisos

# Extração de Dados (Extract)

In [3]:
path_contratos = '/content/drive/MyDrive/python-data-analytics/dados/ tabela_contratos.csv'
path_datas = '/content/drive/MyDrive/python-data-analytics/dados/tabela_datas.csv'
path_empresas = '/content/drive/MyDrive/python-data-analytics/dados/tabela_empresas.csv'

In [4]:
tbcontratos = pd.read_csv(path_contratos)
tbcontratos.head(2)

,id_contrato,nome_contrato,objeto_contrato,valor_contrato,inicio_vigencia,termino_vigencia,fk_empresa_contratada
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,4,32,1
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,13,33,2


In [5]:
tbdatas = pd.read_csv(path_datas)
tbdatas.head(2)

,id_data,data
0,1,20/05/2014
1,2,27/09/2014


In [6]:
tbempresas = pd.read_csv (path_empresas)
tbempresas.head(2)
# tbempresas.tail () -- 5 últimos
# tbempresas.sample () -- 5 aleatórios

,id_empresa,nome_empresa
0,1,Thomas Jefferson
1,2,Chaveiro City


In [7]:
tbcontratos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_contrato            184 non-null    int64  
 1   nome_contrato          184 non-null    object 
 2   objeto_contrato        184 non-null    object 
 3   valor_contrato         184 non-null    float64
 4   inicio_vigencia        184 non-null    int64  
 5   termino_vigencia       184 non-null    int64  
 6   fk_empresa_contratada  184 non-null    int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 10.2+ KB


In [8]:
tbdatas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id_data  254 non-null    int64 
 1   data     254 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.1+ KB


In [9]:
tbempresas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id_empresa    68 non-null     int64 
 1   nome_empresa  68 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.2+ KB


# Transformação de Dados (Transform)

In [10]:
# "JOIN" entre tabela de Contratos e Empresas
tbcontratos_mod = tbcontratos.merge(
    tbempresas,
    left_on= 'fk_empresa_contratada', # Chave da tabela esquerda
    right_on = 'id_empresa', # Chave da tabela da direita
    how ='left' # Todas da esquerda aparecem (tbcontratos) mesmo que não tenham correspondência com a direita
    )
tbcontratos_mod.head(2)

,id_contrato,nome_contrato,objeto_contrato,valor_contrato,inicio_vigencia,termino_vigencia,fk_empresa_contratada,id_empresa,nome_empresa
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,4,32,1,1,Thomas Jefferson
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,13,33,2,2,Chaveiro City


In [11]:
# Excluindo as colunas que não precisam aparecer
tbcontratos_mod = tbcontratos_mod.drop(columns=['fk_empresa_contratada', 'id_empresa'])
tbcontratos_mod.head(2)


,id_contrato,nome_contrato,objeto_contrato,valor_contrato,inicio_vigencia,termino_vigencia,nome_empresa
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,4,32,Thomas Jefferson
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,13,33,Chaveiro City


In [12]:
#"JOIN" das tabelas Contratos e Datas (Inicio Vigencia)
tbcontratos_mod_dti = tbcontratos_mod.merge(
    tbdatas,
    left_on='inicio_vigencia',
    right_on= 'id_data',
    how='left'
)
tbcontratos_mod_dti.head(2)


,id_contrato,nome_contrato,objeto_contrato,valor_contrato,inicio_vigencia,termino_vigencia,nome_empresa,id_data,data
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,4,32,Thomas Jefferson,4,20/12/2014
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,13,33,Chaveiro City,13,26/02/2015


In [13]:
tbcontratos_mod_dti.drop(columns=['id_data','inicio_vigencia'],inplace=True)
#Renomeando coluna data
tbcontratos_mod_dti.rename(columns={'data':'data_inicio_vigencia'},inplace=True)
tbcontratos_mod_dti.head(2)

,id_contrato,nome_contrato,objeto_contrato,valor_contrato,termino_vigencia,nome_empresa,data_inicio_vigencia
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,32,Thomas Jefferson,20/12/2014
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,33,Chaveiro City,26/02/2015


In [14]:
#"JOIN" das tabelas Contratos e Datas (Termino Vigencia)
tbcontratos_final = tbcontratos_mod_dti.merge(
    tbdatas,
    left_on= 'termino_vigencia',
    right_on= 'id_data',
    how='left'
)
tbcontratos_final.head(2)

,id_contrato,nome_contrato,objeto_contrato,valor_contrato,termino_vigencia,nome_empresa,data_inicio_vigencia,id_data,data
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,32,Thomas Jefferson,20/12/2014,32,19/12/2015
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,33,Chaveiro City,26/02/2015,33,31/12/2015


In [15]:
tbcontratos_final.drop(columns=['id_data','termino_vigencia'],inplace=True)
#Renomeando coluna data
tbcontratos_final.rename(columns={'data':'data_termino_vigencia'},inplace=True)
tbcontratos_final.head(2)

,id_contrato,nome_contrato,objeto_contrato,valor_contrato,nome_empresa,data_inicio_vigencia,data_termino_vigencia
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,Thomas Jefferson,20/12/2014,19/12/2015
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,Chaveiro City,26/02/2015,31/12/2015


In [16]:
tbcontratos_final['data_inicio_vigencia'] = pd.to_datetime(tbcontratos_final['data_inicio_vigencia'], format="%d/%m/%Y")

tbcontratos_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id_contrato            184 non-null    int64         
 1   nome_contrato          184 non-null    object        
 2   objeto_contrato        184 non-null    object        
 3   valor_contrato         184 non-null    float64       
 4   nome_empresa           184 non-null    object        
 5   data_inicio_vigencia   184 non-null    datetime64[ns]
 6   data_termino_vigencia  184 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 10.2+ KB


In [18]:
tbcontratos_final['data_termino_vigencia'] .iloc[56]

'31/09/2017'

In [19]:
tbcontratos_final.loc[tbcontratos_final['data_termino_vigencia']=='31/09/2017','data_termino_vigencia']='30/09/2017'

In [20]:
tbcontratos_final['data_termino_vigencia'] = pd.to_datetime(tbcontratos_final['data_termino_vigencia'], format="%d/%m/%Y")

tbcontratos_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id_contrato            184 non-null    int64         
 1   nome_contrato          184 non-null    object        
 2   objeto_contrato        184 non-null    object        
 3   valor_contrato         184 non-null    float64       
 4   nome_empresa           184 non-null    object        
 5   data_inicio_vigencia   184 non-null    datetime64[ns]
 6   data_termino_vigencia  184 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(3)
memory usage: 10.2+ KB


In [21]:
tbcontratos_final['tempo_contrato'] = (tbcontratos_final['data_termino_vigencia'] - tbcontratos_final['data_inicio_vigencia']).dt.days
tbcontratos_final.head()

,id_contrato,nome_contrato,objeto_contrato,valor_contrato,nome_empresa,data_inicio_vigencia,data_termino_vigencia,tempo_contrato
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,Thomas Jefferson,2014-12-20,2015-12-19,364
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,Chaveiro City,2015-02-26,2015-12-31,308
2,3,Disp. Lic. 2,Prestação de serviço na confecção de Carimbos,1800.00,HBL Carimbos,2015-02-10,2015-12-31,324
3,4,32/2014,Serviço de Telefônia Móvel,73915.08,Claro S/A,2015-01-01,2015-12-31,364
4,5,001/2014,Publicações no Diário Oficial da União,45000.00,Imprensa Nacional,2015-01-27,2016-01-26,364


In [22]:
tbcontratos_final.to_excel('Contratos_etl.xlsx',index=False) # Tirar a coluna de índice no excel

In [23]:
path='/content/drive/MyDrive/python-data-analytics/dados/'
tbcontratos_final.to_csv(path+'contrato_etl.csv',index=False)

# Aplicação



In [ ]:
import gradio as gr
import pandas as pd



with gr.Blocks(theme=gr.themes.Base(), title="Ferramenta ETL de Contratos") as app:
  gr.Markdown("# Ferramenta Interativa de ETL de Contratos")

  app.launch(share=False, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>